In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 1.57 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/30_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/30_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/30_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/lstm/'+file[0:7]+'result_30min.csv')

Epoch 1/200
2207/2207 [==============================] - 1s 310us/step - loss: 0.0455
Epoch 2/200
2207/2207 [==============================] - 0s 76us/step - loss: 0.0134
Epoch 3/200
2207/2207 [==============================] - 0s 74us/step - loss: 0.0077
Epoch 4/200
2207/2207 [==============================] - 0s 75us/step - loss: 0.0072
Epoch 5/200
2207/2207 [==============================] - 0s 77us/step - loss: 0.0068
Epoch 6/200
2207/2207 [==============================] - 0s 80us/step - loss: 0.0067
Epoch 7/200
2207/2207 [==============================] - 0s 83us/step - loss: 0.0067
Epoch 8/200
2207/2207 [==============================] - 0s 79us/step - loss: 0.0067
Epoch 9/200
2207/2207 [==============================] - 0s 76us/step - loss: 0.0064
Epoch 10/200
2207/2207 [==============================] - 0s 75us/step - loss: 0.0064
Epoch 11/200
2207/2207 [==============================] - 0s 89us/step - loss: 0.0063
Epoch 12/200
2207/2207 [==============================] - 0s 7

2207/2207 [==============================] - 0s 97us/step - loss: 0.0061
Epoch 96/200
2207/2207 [==============================] - 0s 99us/step - loss: 0.0061
Epoch 97/200
2207/2207 [==============================] - 0s 102us/step - loss: 0.0058
Epoch 98/200
2207/2207 [==============================] - 0s 100us/step - loss: 0.0057
Epoch 99/200
2207/2207 [==============================] - 0s 91us/step - loss: 0.0057
Epoch 100/200
2207/2207 [==============================] - 0s 95us/step - loss: 0.0059
Epoch 101/200
2207/2207 [==============================] - 0s 100us/step - loss: 0.0059
Epoch 102/200
2207/2207 [==============================] - 0s 103us/step - loss: 0.0058
Epoch 103/200
2207/2207 [==============================] - 0s 93us/step - loss: 0.0058
Epoch 104/200
2207/2207 [==============================] - 0s 98us/step - loss: 0.0058
Epoch 105/200
2207/2207 [==============================] - 0s 100us/step - loss: 0.0057
Epoch 106/200
2207/2207 [==============================]

2207/2207 [==============================] - 0s 120us/step - loss: 0.0052
Epoch 189/200
2207/2207 [==============================] - 0s 146us/step - loss: 0.0051
Epoch 190/200
2207/2207 [==============================] - 0s 156us/step - loss: 0.0052
Epoch 191/200
2207/2207 [==============================] - 0s 152us/step - loss: 0.0051
Epoch 192/200
2207/2207 [==============================] - 0s 143us/step - loss: 0.0052
Epoch 193/200
2207/2207 [==============================] - 0s 130us/step - loss: 0.0051
Epoch 194/200
2207/2207 [==============================] - 0s 145us/step - loss: 0.0052
Epoch 195/200
2207/2207 [==============================] - 0s 142us/step - loss: 0.0052
Epoch 196/200
2207/2207 [==============================] - 0s 159us/step - loss: 0.0052
Epoch 197/200
2207/2207 [==============================] - 0s 154us/step - loss: 0.0052
Epoch 198/200
2207/2207 [==============================] - 0s 153us/step - loss: 0.0052
Epoch 199/200
2207/2207 [=====================

2255/2255 [==============================] - 0s 209us/step - loss: 0.0041
Epoch 82/200
2255/2255 [==============================] - 0s 179us/step - loss: 0.0041
Epoch 83/200
2255/2255 [==============================] - 0s 180us/step - loss: 0.0040
Epoch 84/200
2255/2255 [==============================] - 0s 185us/step - loss: 0.0040
Epoch 85/200
2255/2255 [==============================] - 0s 175us/step - loss: 0.0041
Epoch 86/200
2255/2255 [==============================] - 0s 176us/step - loss: 0.0042
Epoch 87/200
2255/2255 [==============================] - 0s 168us/step - loss: 0.0040
Epoch 88/200
2255/2255 [==============================] - 0s 161us/step - loss: 0.0041
Epoch 89/200
2255/2255 [==============================] - 0s 177us/step - loss: 0.0041
Epoch 90/200
2255/2255 [==============================] - 0s 184us/step - loss: 0.0040
Epoch 91/200
2255/2255 [==============================] - 0s 178us/step - loss: 0.0042
Epoch 92/200
2255/2255 [==============================] 

2255/2255 [==============================] - 0s 209us/step - loss: 0.0038
Epoch 175/200
2255/2255 [==============================] - 0s 196us/step - loss: 0.0038
Epoch 176/200
2255/2255 [==============================] - 0s 177us/step - loss: 0.0037
Epoch 177/200
2255/2255 [==============================] - 0s 183us/step - loss: 0.0037
Epoch 178/200
2255/2255 [==============================] - 0s 192us/step - loss: 0.0036
Epoch 179/200
2255/2255 [==============================] - 0s 176us/step - loss: 0.0037
Epoch 180/200
2255/2255 [==============================] - 0s 166us/step - loss: 0.0036
Epoch 181/200
2255/2255 [==============================] - 0s 173us/step - loss: 0.0036
Epoch 182/200
2255/2255 [==============================] - 0s 169us/step - loss: 0.0037
Epoch 183/200
2255/2255 [==============================] - 0s 189us/step - loss: 0.0036
Epoch 184/200
2255/2255 [==============================] - 0s 188us/step - loss: 0.0036
Epoch 185/200
2255/2255 [=====================

...

1849/1849 [==============================] - 0s 240us/step - loss: 0.0051
Epoch 147/200
1849/1849 [==============================] - 0s 256us/step - loss: 0.0053
Epoch 148/200
1849/1849 [==============================] - 0s 265us/step - loss: 0.0051
Epoch 149/200
1849/1849 [==============================] - 0s 264us/step - loss: 0.0050
Epoch 150/200
1849/1849 [==============================] - 0s 253us/step - loss: 0.0053
Epoch 151/200
1849/1849 [==============================] - 0s 239us/step - loss: 0.0050
Epoch 152/200
1849/1849 [==============================] - 0s 245us/step - loss: 0.0052
Epoch 153/200
1849/1849 [==============================] - 0s 235us/step - loss: 0.0051
Epoch 154/200
1849/1849 [==============================] - 1s 272us/step - loss: 0.0051
Epoch 155/200
1849/1849 [==============================] - 1s 288us/step - loss: 0.0051
Epoch 156/200
1849/1849 [==============================] - 1s 317us/step - loss: 0.0049
Epoch 157/200
1849/1849 [=====================

2111/2111 [==============================] - 0s 224us/step - loss: 0.0036
Epoch 39/200
2111/2111 [==============================] - 1s 257us/step - loss: 0.0037
Epoch 40/200
2111/2111 [==============================] - 0s 213us/step - loss: 0.0036
Epoch 41/200
2111/2111 [==============================] - 1s 284us/step - loss: 0.0037
Epoch 42/200
2111/2111 [==============================] - 1s 242us/step - loss: 0.0041
Epoch 43/200
2111/2111 [==============================] - 1s 289us/step - loss: 0.0036
Epoch 44/200
2111/2111 [==============================] - 0s 233us/step - loss: 0.0040 0s - loss
Epoch 45/200
2111/2111 [==============================] - 1s 260us/step - loss: 0.0036
Epoch 46/200
2111/2111 [==============================] - 1s 294us/step - loss: 0.0046
Epoch 47/200
2111/2111 [==============================] - 0s 228us/step - loss: 0.0036
Epoch 48/200
2111/2111 [==============================] - 0s 207us/step - loss: 0.0040
Epoch 49/200
2111/2111 [======================

2111/2111 [==============================] - 0s 217us/step - loss: 0.0034
Epoch 132/200
2111/2111 [==============================] - 1s 345us/step - loss: 0.0035
Epoch 133/200
2111/2111 [==============================] - 1s 237us/step - loss: 0.0034
Epoch 134/200
2111/2111 [==============================] - 1s 261us/step - loss: 0.0041
Epoch 135/200
2111/2111 [==============================] - 1s 296us/step - loss: 0.0033
Epoch 136/200
2111/2111 [==============================] - 1s 264us/step - loss: 0.0037
Epoch 137/200
2111/2111 [==============================] - 0s 230us/step - loss: 0.0033
Epoch 138/200
2111/2111 [==============================] - 0s 215us/step - loss: 0.0033
Epoch 139/200
2111/2111 [==============================] - 1s 240us/step - loss: 0.0032
Epoch 140/200
2111/2111 [==============================] - 0s 210us/step - loss: 0.0033
Epoch 141/200
2111/2111 [==============================] - 0s 232us/step - loss: 0.0032
Epoch 142/200
2111/2111 [=====================

2184/2184 [==============================] - 1s 256us/step - loss: 0.0059
Epoch 24/200
2184/2184 [==============================] - 1s 261us/step - loss: 0.0058
Epoch 25/200
2184/2184 [==============================] - 1s 286us/step - loss: 0.0061
Epoch 26/200
2184/2184 [==============================] - 0s 211us/step - loss: 0.0056
Epoch 27/200
2184/2184 [==============================] - 1s 281us/step - loss: 0.0056
Epoch 28/200
2184/2184 [==============================] - 0s 218us/step - loss: 0.0058
Epoch 29/200
2184/2184 [==============================] - 0s 213us/step - loss: 0.0058
Epoch 30/200
2184/2184 [==============================] - 0s 212us/step - loss: 0.0054
Epoch 31/200
2184/2184 [==============================] - 0s 225us/step - loss: 0.0054
Epoch 32/200
2184/2184 [==============================] - 1s 232us/step - loss: 0.0057
Epoch 33/200
2184/2184 [==============================] - 1s 262us/step - loss: 0.0058
Epoch 34/200
2184/2184 [==============================] 

2184/2184 [==============================] - 1s 268us/step - loss: 0.0048
Epoch 118/200
2184/2184 [==============================] - 1s 239us/step - loss: 0.0045
Epoch 119/200
2184/2184 [==============================] - 0s 215us/step - loss: 0.0047
Epoch 120/200
2184/2184 [==============================] - 1s 233us/step - loss: 0.0047
Epoch 121/200
2184/2184 [==============================] - 1s 235us/step - loss: 0.0047
Epoch 122/200
2184/2184 [==============================] - 0s 222us/step - loss: 0.0046
Epoch 123/200
2184/2184 [==============================] - 0s 197us/step - loss: 0.0046
Epoch 124/200
2184/2184 [==============================] - 0s 212us/step - loss: 0.0044
Epoch 125/200
2184/2184 [==============================] - 0s 217us/step - loss: 0.0046
Epoch 126/200
2184/2184 [==============================] - 0s 209us/step - loss: 0.0047
Epoch 127/200
2184/2184 [==============================] - 0s 203us/step - loss: 0.0046
Epoch 128/200
2184/2184 [=====================

In [4]:
rmse_list

{'584-ws-training.csv': 28.23636345109497,
 '567-ws-training.csv': 26.135379941403674,
 '596-ws-training.csv': 20.279161733661752,
 '552-ws-training.csv': 21.144248884527034,
 '544-ws-training.csv': 20.942708934324845,
 '540-ws-training.csv': 29.15271362920339}

time: 9.68 ms


In [5]:
mae_list

{'584-ws-training.csv': 20.428650167359454,
 '567-ws-training.csv': 17.333119168673928,
 '596-ws-training.csv': 14.930143259684243,
 '552-ws-training.csv': 14.850284468970639,
 '544-ws-training.csv': 14.538497840033637,
 '540-ws-training.csv': 22.32623531095202}

time: 4.77 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 24.31509609570261
Standard Deviation - RMSE : 3.647107249345429
Mean - MAE : 17.401155035945653
Standard Deviation - MAE : 3.006504543885233
time: 4.47 ms
